In [65]:
import numpy as np
import pandas as pd
from sqlalchemy import create_engine, inspect
import sqlalchemy as sql


In [33]:
from plotly import graph_objs as go

In [47]:
engine = create_engine('sqlite:///DisasterResponse.db')
df = pd.read_sql_table('data/DisasterResponse.db', con=engine)
df.head()

,id,message,original,genre,related,request,offer,aid_related,medical_help,medical_products,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct,1,0,0,1,0,0,...,0,0,1,0,1,0,0,0,0,0
2,8,Looking for someone but no name,"Patnm, di Maryani relem pou li banm nouvel li ...",direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,9,UN reports Leogane 80-90 destroyed. Only Hospi...,UN reports Leogane 80-90 destroyed. Only Hospi...,direct,1,1,0,1,0,1,...,0,0,0,0,0,0,0,0,0,0
4,12,"says: west side of Haiti, rest of the country ...",facade ouest d Haiti et le reste du pays aujou...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [27]:
categories = df.columns[4:].values
counts = df[df.columns[4:]]

array(['related', 'request', 'offer', 'aid_related', 'medical_help',
       'medical_products', 'search_and_rescue', 'security', 'military',
       'child_alone', 'water', 'food', 'shelter', 'clothing', 'money',
       'missing_people', 'refugees', 'death', 'other_aid',
       'infrastructure_related', 'transport', 'buildings', 'electricity',
       'tools', 'hospitals', 'shops', 'aid_centers',
       'other_infrastructure', 'weather_related', 'floods', 'storm',
       'fire', 'earthquake', 'cold', 'other_weather', 'direct_report'],
      dtype=object)

In [34]:
categpry_counts = df[df.columns[4:]].sum().reset_index()
category_counts

,index,0
0,related,20094
1,request,4474
2,offer,118
3,aid_related,10860
4,medical_help,2084
5,medical_products,1313
6,search_and_rescue,724
7,security,471
8,military,860
9,child_alone,0


In [38]:
graph_one = [go.Bar(
    x = counts['index'],
    y = counts[0]
)]

In [39]:
graph_one

[Bar({
     'x': array(['related', 'request', 'offer', 'aid_related', 'medical_help',
                 'medical_products', 'search_and_rescue', 'security', 'military',
                 'child_alone', 'water', 'food', 'shelter', 'clothing', 'money',
                 'missing_people', 'refugees', 'death', 'other_aid',
                 'infrastructure_related', 'transport', 'buildings', 'electricity',
                 'tools', 'hospitals', 'shops', 'aid_centers', 'other_infrastructure',
                 'weather_related', 'floods', 'storm', 'fire', 'earthquake', 'cold',
                 'other_weather', 'direct_report'], dtype=object),
     'y': array([20094,  4474,   118, 10860,  2084,  1313,   724,   471,   860,     0,
                  1672,  2923,  2314,   405,   604,   298,   875,  1194,  3446,  1705,
                  1201,  1333,   532,   159,   283,   120,   309,  1151,  7297,  2155,
                  2443,   282,  2455,   530,  1376,  5075], dtype=int64)
 })]

In [ ]:
    graphs = [
        {
            'data': [
                Bar(
                    x=genre_names,
                    y=genre_counts
                )
            ],

            'layout': {
                'title': 'Distribution of Message Genres',
                'yaxis': {
                    'title': "Count"
                },
                'xaxis': {
                    'title': "Genre"
                }
            },
        },
        {
            'data': [
                Bar(
                    x = category_counts['index'],
                    y = category_counts[0]
                )
            ],
            'layout': {
                'title': 'Distribution of Message Categories',
                'yaxis': {
                    'title': "Count"
                },
                'xaxis': {
                    'title': "Category"
                }
            }            
            
        }
    ]

In [138]:
import re
import nltk
nltk.download('stopwords')
from nltk.tokenize import RegexpTokenizer
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords


def tokenize(text):
    ''' Converts text into tokens for ML processing
    INPUT: text: a string with a series of words, ideally in natural language.
    OUTPUT: clean_tokens: a list of tokens extracted from the original text.
    '''
    url_regex = 'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+'
    detected_urls = re.findall(url_regex, text)
    for url in detected_urls:
        text = text.replace(url, 'urlplaceholder')

    tokens = RegexpTokenizer("[a-zA-Z0-9'-]+").tokenize(text)
    lemmatizer = WordNetLemmatizer()
    stop_words = set(stopwords.words('english'))

    clean_tokens = []
    for tok in tokens:
        if tok not in stop_words:
            clean_tok = lemmatizer.lemmatize(tok).lower().strip()
            clean_tokens.append(clean_tok)
        
    return clean_tokens

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Felipe\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [143]:
def generate_best_tokens(df):
    ''' Tokenizes the message column of the dataframe, adds it as a row to a new dataframe,
        and then outputs the top 500 most used tokens based on that dataframe.
        INPUT: df: a pandas DataFrame object
        OUTPUT: best_tokens: an array of the top 500 most used tokens in df.
    '''
    token_dict = {}
    tokens_set = set()
    index = 0
    for message in df.message:
        token_dict[index] = tokenize(message)
        tokens_set.update(token_dict[index])
        index += 1
    token_df = pd.DataFrame(index=np.arange(0, len(tokens_set)), columns=tokens_set)    
    for i in token_dict:
        for token in token_dict[i]:
            token_df.iloc[i][token] = 1
    best_tokens = token_df.sum().sort_values(ascending=False)[:500]
    return best_tokens

In [141]:
best_tokens = df2.sum().sort_values(ascending=False)[:500]

In [144]:
best_tokens.reset_index()

,index,0
0,i,4173
1,the,3651
2,people,2623
3,food,2577
4,water,2459
...,...,...
495,called,142
496,monday,141
497,percent,141
498,start,141


In [139]:
tokenize(df.message[1])

['is', 'hurricane']